# Empilement des images calibrées avec option d'alignement

Ce notebook :
- Calibre les images de chaque filtre (g, r, i)
- Soustrait le bias et le dark, divise par le flat
- Aligne les images si `align_images = True`
- Combine les images par médiane pour créer un stack final

In [ ]:
import numpy as np
from astropy.io import fits
from pathlib import Path
from skimage.registration import phase_cross_correlation
from scipy.ndimage import shift
import matplotlib.pyplot as plt

# -----------------------------
# Paramètres utilisateur
# -----------------------------
filters = ['g', 'r', 'i']
align_images = True  # activer/désactiver l'alignement

# Chemins
bias_path = Path('master_bias_23.fits')
darks_dir = Path('master_darks_23/')
flats_dir = Path('master_flats_lyon')
raw_dir = Path('/Volumes/Astro/ohp/CANAGUIER_LOUSTALET_SAPHORES/test')  # contient g/, r/, i/
output_dir = Path('stacks/')
output_dir.mkdir(exist_ok=True)

In [ ]:
# Chargement des masters
master_bias = fits.getdata(bias_path).astype(np.float32)

def load_master_dark(exptime):
    path = darks_dir / f'master_dark_{exptime}s.fits'
    if path.exists():
        return fits.getdata(path).astype(np.float32)
    else:
        print(f"master_dark_{exptime}s.fits introuvable, dark ignoré.")
        return np.zeros_like(master_bias)

def load_master_flat(filter_name):
    path = flats_dir / f'master_flat_{filter_name}.fits'
    if path.exists():
        return fits.getdata(path).astype(np.float32)
    else:
        raise FileNotFoundError(f"Master flat manquant pour {filter_name}")

# Application des calibrations
$OUTPUT = \frac{INPUT - DARK - BIAS}{FLAT}$

In [ ]:
# Boucle sur les filtres
for f in filters:
    subdir = raw_dir / f
    files = sorted(subdir.glob('M*.fits'))
    if len(files) == 0:
        print(f"Aucun fichier pour le filtre {f}")
        continue

    print(f"\nFiltre {f} — {len(files)} images")

    master_flat = load_master_flat(f)
    calibrated_images = []
    ref_image = None

    for i, file in enumerate(files):
        with fits.open(file, ignore_missing_end=True) as hdul:
            data = hdul[0].data.astype(np.float32)
            header = hdul[0].header
            exptime = int(hdul[0].header.get('EXPTIME', None))
            
            # Calibration basique
            master_dark = load_master_dark(5) if exptime else np.zeros_like(data)
            calibrated = (data - master_bias - master_dark) #/ master_flat ici exceptionnellement on a pas de flats

            # Alignement
            if align_images:
                if ref_image is None:
                    ref_image = calibrated
                    calibrated_images.append(calibrated)
                    continue
                shift_estimate, error, _ = phase_cross_correlation(ref_image, calibrated, upsample_factor=10)
                aligned = shift(calibrated, shift_estimate)
                calibrated_images.append(aligned)
            else:
                calibrated_images.append(calibrated)

    stack = np.median(np.array(calibrated_images), axis=0)
    out_path = output_dir / f'stack_{f}.fits'
    fits.writeto(out_path, stack.astype(np.float32), overwrite=True, header=header)
    print(f"Stack sauvegardé : {out_path}")

    plt.figure(figsize=(6,5))
    plt.imshow(stack, origin='lower', cmap='gray',
               vmin=np.percentile(stack, 5), vmax=np.percentile(stack, 95))
    plt.title(f'Stack {f} ({'aligné' if align_images else 'non aligné'})')
    plt.colorbar()
    plt.show()